In [1]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
#from datetime import datetime, date, timedelta
from selenium.webdriver.chrome.options import Options
import requests
import json

from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os

import datetime
import unicodedata

from dotenv import load_dotenv
load_dotenv(".env")

True

In [55]:
import sshtunnel
from sshtunnel import SSHTunnelForwarder

server = sshtunnel.SSHTunnelForwarder((os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
    ssh_pkey="sercret/akasaka.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )) 
# SSH接続確認


# 現在時刻
today = datetime.datetime.now()
print(today)
#[結果] 2021-08-23 07:12:20.806648
# 1日後
today_str:str = today.strftime('%Y-%m-%d')
eight_days_after:str = (today + datetime.timedelta(days=8)).strftime('%Y-%m-%d')
yesterday:str = (today + datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
#### Create dataframe from resultant table ####

server.start()

print(f"local bind port: {server.local_bind_port}")
# データベース接続
cnx = mysql.connector.connect(
    host="localhost", 
    port=server.local_bind_port, 
    user=os.getenv('WORDPRESS_DB_ID'), 
    password=os.getenv('DB_PASSWORD'), 
    database=os.getenv('WORDPRESS_DB_NAME'), 
    charset='utf8',
    use_pure=True
    )

# 接続確認
print(f"sql connection status: {cnx.is_connected()}")
cursor = cnx.cursor()

2023-06-28 22:16:40.180137
local bind port: 64506
sql connection status: True


### テーブルの数を数える

In [56]:
cursor.execute("show tables;")
myresult = cursor.fetchall()
 
for x in myresult:
  print(x[0])

maptable
parlar_data
pledge
schedule


### それぞれのテーブルの確認

In [58]:
for table_name in myresult:
        print(table_name[0])
        sql = f"""
SELECT *
FROM {table_name[0]}
limit 10"""
        print(sql)
        cursor.execute(sql)
        #cols = [col[0] for col in cursor.description]
        cols = cursor.column_names
        convart_table_df = pd.DataFrame(cursor.fetchall(),columns = cols)
        display(convart_table_df.head())

maptable

SELECT *
FROM maptable
limit 10


,parlar_map_id,都道府県,kintone_id,shop_id,ホールナビ店舗名,DMM店舗名,アナスロ店舗名,緯度,経度,住所,取得時間
0,46,北海道,P01378,1378,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,43.0858,141.515,北海道江別市大麻泉町47-37,2023-03-18 05:36:38
1,47,北海道,P01582,1582,ひまわり音更店,音更ひまわり,ひまわり音更店,42.9614,143.207,北海道河東郡音更町木野大通東13丁目1番21,2023-03-18 05:36:38
2,48,北海道,P01351,13160,EBH,EBH,EBH,43.1063,141.293,北海道札幌市西区発寒15条4-13-1,2023-03-18 05:36:38
3,49,茨城県,P02562,2562,キコーナ水戸駅南店,キコーナ水戸駅南店,キコーナ水戸駅南店,36.3655,140.477,茨城県水戸市城南1-4-1,2023-03-18 05:36:38
4,50,神奈川県,P04933,4933,ピーズ・タウン桜ヶ丘,P’Stown桜ヶ丘,ピーズ・タウン桜ヶ丘,35.4511,139.465,神奈川県大和市福田1-6-10,2023-03-18 05:36:39


parlar_data

SELECT *
FROM parlar_data
limit 10


,店舗名,日付,Nのつく日,都道府県,機種名,台番号,G数,差枚,BB,RB,ART,BB確率,RB確率,ART確率,合成確率,id
0,123+N東雲店,2023-05-24,4,東京都,マイジャグラーV,1034,1397,87,6,2,0,1/232.8,1/698.5,1/0.0,1/174.6,15180472
1,123+N東雲店,2023-05-24,4,東京都,マイジャグラーV,1035,731,-448,1,1,0,1/731.0,1/731.0,1/0.0,1/365.5,15180473
2,123+N東雲店,2023-05-24,4,東京都,マイジャグラーV,1036,3935,831,17,11,0,1/231.5,1/357.7,1/0.0,1/140.5,15180474
3,123+N東雲店,2023-05-24,4,東京都,マイジャグラーV,1037,2815,2953,23,8,0,1/122.4,1/351.9,1/0.0,1/90.8,15180475
4,123+N東雲店,2023-05-24,4,東京都,マイジャグラーV,1062,3770,-576,12,10,0,1/314.2,1/377.0,1/0.0,1/171.4,15180476


pledge

SELECT *
FROM pledge
limit 10


,id,取材名,媒体名,公約内容,取得時間
0,1,ｙｔｒ来店実践,SEVENSTV,来店実践,2023-03-16 03:56:31
1,2,Ａレポート,アツ姫,Aタイプに高設定投入,2023-03-16 03:56:31
2,3,黒バラの極意(来店ver),ジャンバリ.TV,,2023-03-16 03:56:31
3,4,鮮魚レポート,アメスロ,,2023-03-16 03:56:31
4,5,魔法少女レポート スロット取材,爆ガチ！,まどかマギカシリーズが狙い目,2023-03-16 03:56:31


schedule

SELECT *
FROM schedule
limit 10


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,67,長野県,2023-03-15,水,ニューアサヒ長野青木島店,ぱちレポA,APチャンネル,C,2023-03-15
1,68,岐阜県,2023-03-15,水,ウイング池田南店,ジャックレポート,APチャンネル,・,2023-03-15
2,69,長野県,2023-03-16,木,チャンピオンANNEX,APチャンネル来店収録,APチャンネル,B,2023-03-15
3,70,三重県,2023-03-16,木,サン亀山店,ジャックレポート,APチャンネル,・,2023-03-15
4,71,三重県,2023-03-17,金,サン亀山店,沖花レポート,APチャンネル,・,2023-03-15


In [ ]:
cursor.execute("show tables;")
myresult = cursor.fetchall()
 
for x in myresult:
  print(x)

In [34]:
sql = f"""
        SELECT *
        FROM pledge
        """
print(sql)
cursor.execute(sql)
#cols = [col[0] for col in cursor.description]
sql_syuzai_report_all_df = pd.DataFrame(cursor.fetchall(),columns = ['id','取材名','媒体名','公約内容','取得時間'])
sql_syuzai_report_all_df


        SELECT *
        FROM pledge
        


,id,取材名,媒体名,公約内容,取得時間
0,1,ｙｔｒ来店実践,SEVENSTV,来店実践,2023-03-16 03:56:31
1,2,Ａレポート,アツ姫,Aタイプに高設定投入,2023-03-16 03:56:31
2,3,黒バラの極意(来店ver),ジャンバリ.TV,,2023-03-16 03:56:31
3,4,鮮魚レポート,アメスロ,,2023-03-16 03:56:31
4,5,魔法少女レポート スロット取材,爆ガチ！,まどかマギカシリーズが狙い目,2023-03-16 03:56:31
...,...,...,...,...,...
2126,101375,しおねえ来店,エースプロ,None,2023-06-17 09:30:52
2127,101376,星のパチは友を呼ぶ,パチとスロは友を呼ぶ,None,2023-06-18 09:25:03
2128,101377,シーサ。の法則,P-martTV,None,2023-06-24 01:48:28
2129,101378,ジョージ来店,その他取材・企画,None,2023-06-24 01:48:28


In [35]:
print(cursor.execute(sql))

None
